# FSA

> In this example, we will scrape the addresses for Deliveroo's FSA hygiene rated kitchens and then search them within Deliveroo to find all the restaurants based at or near these locations.

In [ ]:
#| default_exp fsa

In [1]:
import requests 
from bs4 import BeautifulSoup
from nbdev.showdoc import *
from deliveroo_editions.selenium_utils import *
from deliveroo_editions.deliveroo_utils import *
import pandas as pd
import datetime

[WDM] - Downloading: 100%|████████████████████████████████████████| 8.29M/8.29M [00:00<00:00, 9.70MB/s]


Deliveroo may not be at Aviemore Centre, Aviemore Centre, Aviemore PH22 1PF, UK yet.
Deliveroo may not be at AB3 9HR yet.


Lets load all districts into a csv:

In [2]:
districts = pd.read_csv('data/districts.csv')

In [3]:
districts

,Postcode,Latitude,Longitude,Easting,Northing,Grid Reference,Town/Area,Region,Postcodes,Active postcodes,Population,Households,Nearby districts,UK region,Post Town
0,AB1,57.1269,-2.136440,391838.0,804005.0,NJ918040,Aberdeen,Aberdeen,2655.0,0.0,NaN,NaN,"AB10, AB9, AB15, AB25, AB2, AB12, AB16, AB11, ...",Scotland,NaN
1,AB2,57.1713,-2.141520,391541.0,808949.0,NJ915089,Aberdeen,Aberdeen,3070.0,0.0,NaN,NaN,"AB16, AB22, AB24, AB25, AB1, AB9, AB15, AB10, ...",Scotland,NaN
2,AB3,57.0876,-2.596240,363963.0,799780.0,NO639997,Aberdeen,Aberdeen,2168.0,0.0,NaN,NaN,"AB31, AB34, AB52, AB33, AB14, AB32, AB30, AB13...",Scotland,NaN
3,AB4,57.5343,-2.127130,392487.0,849358.0,NJ924493,"Fraserburgh, Peterhead, Macduff",Aberdeen,2956.0,0.0,NaN,NaN,"AB41, AB43, AB42, AB53, AB22, AB23, AB2, AB24,...",Scotland,NaN
4,AB5,57.4652,-2.647640,361248.0,841843.0,NJ612418,"Buckie, Huntly, Inverurie",Aberdeen,3002.0,0.0,NaN,NaN,"AB54, AB52, AB45, AB53, AB33, AB44, AB55, AB51...",Scotland,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,YO91,53.9779,-1.066370,461330.0,453941.0,SE613539,"Lawrence Court, PO Boxes",York,10.0,0.0,NaN,NaN,"YO31, YO3, YO1, YO10, YO32, YO90, YO30, YO24, ...",Yorkshire and The Humber,York
3110,YO95,53.9267,-0.815105,477904.0,448491.0,SE779484,Pocklington,York,1.0,0.0,NaN,NaN,"YO42, YO4, YO41, YO43, YO19, YO17, YO10, YO1, ...",Yorkshire and The Humber,NaN
3111,ZE1,60.1517,-1.168180,446285.0,1141092.0,HU462410,"Lerwick, Scalloway",Shetland,393.0,327.0,8445.0,3862.0,"ZE2, ZE3, KW17, KW15, KW16, AB42, AB43, AB4, K...",Scotland,Shetland
3112,ZE2,60.3080,-1.229190,442695.0,1158458.0,HU426584,Shetland,Shetland,347.0,313.0,14209.0,5845.0,"ZE1, ZE3, KW17, KW15, KW16, AB42, AB43, AB4, K...",Scotland,Shetland


To find if Deliveroo delivers to a postcode district, we need to provide a postcode in the district to search Deliveroo with. 

In [4]:
def get_central_postcode(district:str):
    url = "https://api.postcodes.io/outcodes/" + district
    response = requests.get(url).json()
    if response['status'] == 200:
        lon = str(response['result']['longitude'])
        lat = str(response['result']['latitude'])
        geo_url = "https://api.postcodes.io/postcodes?lon=" + lon + "&lat=" + lat + "&widesearch=5000&limit=1"
        geo_response = requests.get(geo_url).json()
        if geo_response['status'] == 200:
            if geo_response['result']: 
                postcode = geo_response['result'][0]['postcode']
                return postcode
            else:
                print("No postcodes at coordinates")
                return
        else:
            print(f"Search for coords: {lat}, {lon} returned: {geo_response['status']}")
            return
    else:
        print(f"Search for district: {district} returned: {response['status']}")
        return

In [5]:
get_central_postcode("AB1")

Search for district: AB1 returned: 404


In [6]:
address_strings = [get_central_postcode("B3")]

Now we have the ~30 addresses, we can search Deliveroo's site for all the editions kitchens based at these addresses:

In [7]:
from tqdm import tqdm

In [10]:
district_editions = {}
driver = initialise_driver(service,False)
for district in tqdm(districts['Postcode'][2388:]):
    print(district)
    editions_locations = []
    postcode = get_central_postcode(district)
    if postcode:
        attempts = 0
        while attempts < 2:
            try:
                editions_locations = get_editions_locations_near_addresses([postcode], driver=driver)
                district_editions[district] = editions_locations
                attempts = 2
            except:
                attempts += 1    

  0%|                                                   | 0/726 [00:00<?, ?it/s]

S61


  0%|                                         | 1/726 [00:15<3:03:16, 15.17s/it]

S62


  0%|                                         | 2/726 [00:29<2:59:11, 14.85s/it]

...

Deliveroo may not be at ZE1 0SY yet.
ZE2


100%|██████████████████████████████████████▉| 725/726 [1:48:38<00:03,  3.99s/it]

Deliveroo may not be at ZE2 9LP yet.
ZE3


100%|███████████████████████████████████████| 726/726 [1:48:42<00:00,  8.98s/it]

Deliveroo may not be at ZE3 9JL yet.


In [12]:
import json 
def save_dict_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file)

save_dict_to_json(district_editions, 'data/s60_to_ze3.json')


In [24]:
import json

# district_editions = json.loads('data/bt60_to_s60.json')

with open('data/b1_to_bt56.json') as json_file:
    # Load JSON data into a Python dictionary
    district_editions = json.load(json_file)

In [32]:
editions_names

['london/bsy-1-editions',
 'london/culvert-place-editions',
 'london/bsy-2-editions',
 'london/south-bermondsey-area',
 'london/queenstown-road',
 'london/honor-oak-park',
 'london/fish-island-area',
 'london/battersea-york-road',
 'london/south-norbury',
 'london/balham',
 'london/nine-elms',
 'london/wandsworth-editions',
 'london/windsor-and-eton',
 'london/beddington-east',
 'london/maida-vale-editions',
 'london/caledonian-road-and-barnsbury',
 'london/whitechapel-editions',
 'london/acton-editions',
 'london/hammersmith',
 'london/swiss-cottage-editions',
 'london/mornington-crescent-south',
 'london/white-city-estate',
 'london/south-acton',
 'london/brent-cross-editions',
 'wigan/sankey',
 'manchester/sale',
 'cambridge/cambridge-editions',
 'cambridge/cambridge-south',
 'london/canning-town-editions',
 'london/canary-wharf',
 'london/blackwall',
 'london/hornsey-station',
 'london/kentish-town',
 'edinburgh/new-town',
 'edinburgh/broughton',
 'london/jubilee-park-lee-valley',


In [25]:
for key in district_editions:
    for item in district_editions[key]:
        if item not in editions_names:
            editions_names.append(item)

In [30]:
with open('editions_list.txt', 'w') as file:
    # Iterate over the list and write each item to a new line in the file
    for item in editions_names:
        file.write(item + '\n')

In [35]:
with open('data/editions_list.txt', 'r') as file:
    editions_names = file.readlines()
    editions_names = [line.strip() for line in lines] 

In [38]:
editions_names

['birmingham/birmingham-city-centre',
 'brighton/brighton-and-hove-editions',
 'brighton/brighton-centre',
 'brighton/round-hill',
 'bristol/bristol-editions-site',
 'bristol/montpelier-station',
 'cambridge/cambridge-editions',
 'cambridge/cambridge-south',
 'edinburgh/broughton',
 'edinburgh/new-town',
 'glasgow/glasgow-city-centre',
 'glasgow/glasgow-editions',
 'leeds/editions-in-leeds',
 'leeds/wortley',
 'london/acton-editions',
 'london/balham',
 'london/battersea-york-road',
 'london/beddington-east',
 'london/blackwall',
 'london/brent-cross-editions',
 'london/bsy-1-editions',
 'london/bsy-2-editions',
 'london/caledonian-road-and-barnsbury',
 'london/canary-wharf',
 'london/canning-town-editions',
 'london/culvert-place-editions',
 'london/fish-island-area',
 'london/hammersmith',
 'london/honor-oak-park',
 'london/hornsey-station',
 'london/jubilee-park-lee-valley',
 'london/kentish-town',
 'london/maida-vale-editions',
 'london/mornington-crescent-south',
 'london/nine-elm

We can now get all the editions kitchens based at all of these sites:

In [7]:
editions = get_restaurants_from_editions_location(editions_names)

In [8]:
editions_df = pd.DataFrame(editions)

In [9]:
columns_subset = ['name', 'location', 'edition']

In [10]:
editions_df = editions_df.drop_duplicates(columns_subset).reset_index(drop=True)

In [11]:
current_time = datetime.datetime.now()

In [12]:
filename = 'data/editions_' + current_time.strftime("%Y_%m_%d_%Hhr") + ".csv"

In [13]:
editions_df.to_csv(filename, encoding="utf-8")

In [14]:
filename

'data/editions_2023_07_18_14hr.csv'

In [15]:
editions_df

,name,location,edition,restaurant_url
0,Yacob's Kitchen,London,bsy-1-editions,https://deliveroo.co.uk/menu/London/bsy-1-edit...
1,Shake Shack – Burgers & Shakes,London,bsy-1-editions,https://deliveroo.co.uk/menu/London/bsy-1-edit...
2,Poke Shack,London,bsy-1-editions,https://deliveroo.co.uk/menu/London/bsy-1-edit...
3,Dishoom,London,bsy-2-editions,https://deliveroo.co.uk/menu/London/bsy-2-edit...
4,WING·STOP,London,bsy-2-editions,https://deliveroo.co.uk/menu/London/bsy-2-edit...
...,...,...,...,...
220,Nomi Katsu Editions,Bristol,bristol-editions-site,https://deliveroo.co.uk/menu/Bristol/bristol-e...
221,Poke Fusion,Bristol,bristol-editions-site,https://deliveroo.co.uk/menu/Bristol/bristol-e...
222,The Athenian - Vegan Greek Kitchen,Bristol,bristol-editions-site,https://deliveroo.co.uk/menu/Bristol/bristol-e...
223,TRIP CBD Store,Bristol,bristol-editions-site,https://deliveroo.co.uk/menu/Bristol/bristol-e...


In [23]:
[item for item in editions_names if item.split('/')[1] not in editions_df['edition'].unique()]

['london/fish-island-area',
 'london/balham',
 'london/windsor-and-eton',
 'london/swiss-cottage-editions',
 'london/mornington-crescent-south',
 'london/white-city-estate',
 'london/south-acton',
 'manchester/salford-crescent',
 'brighton/round-hill']

In [25]:
new_rows = [{'name': 'Alcohol Delivery London- Park Lane', 'location': 'London', 'edition': 'fish-island-area', 'restaurant_url': 'https://deliveroo.co.uk/menu/London/fish-island-area/bow-road-express'},
           {'name': 'TRIP CBD Store - Balham', 'location': 'London', 'edition': 'balham', 'restaurant_url': 'https://deliveroo.co.uk/menu/london/balham/trip-cbd-store-balham'},
           {'name': 'Saucybird - Windsor and Eton', 'location': 'London', 'edition': 'windsor-and-eton', 'restaurant_url': 'https://deliveroo.co.uk/menu/London/windsor-and-eton/saucybird/'},
           {'name': "Ben & Jerry's Store - Swiss Cottage", 'location': 'London', 'edition': 'swiss-cottage-editions', 'restaurant_url': 'https://deliveroo.co.uk/menu/London/swiss-cottage-editions/ben-and-jerrys-store-editions-bel'},
           {'name': "YUN Restaurant 云南正宗过桥米线 - Camden", 'location': 'London', 'edition': 'mornington-crescent-south', 'restaurant_url': 'https://deliveroo.co.uk/menu/London/mornington-crescent-south/yun-restaurant-east-central'},
           {'name': "Fit Food - White City", 'location': 'London', 'edition': 'white-city-estate', 'restaurant_url': 'https://deliveroo.co.uk/menu/London/white-city-estate/fit-food-white-city'},
           {'name': "TRIP CBD Store - Chiswick", 'location': 'London', 'edition': 'south-acton', 'restaurant_url': 'https://deliveroo.co.uk/menu/london/south-acton/trip-cbd-store-chiswick'},
           {'name': "Halloween Pies - Salford Crescent", 'location': 'Manchester', 'edition': 'salford-crescent', 'restaurant_url': 'https://deliveroo.co.uk/menu/Manchester/salford-crescent/halloween-pies-pr-activation-man'},
           {'name': "The Snacking & Breakfast Store", 'location': 'Manchester', 'edition': 'salford-crescent', 'restaurant_url': 'https://deliveroo.co.uk/menu/Manchester/salford-crescent/kelloggs-snacking-store'},
           {'name': "Lost Boys Chicken @ The Joker - Preston", 'location': 'Brighton', 'edition': 'round-hill', 'restaurant_url': 'https://deliveroo.co.uk/menu/Brighton/round-hill/lost-boys-chicken'},
           {'name': "Saucybird - Brighton", 'location': 'Brighton', 'edition': 'round-hill', 'restaurant_url': 'https://deliveroo.co.uk/menu/Brighton/round-hill/saucybird-brighton'}]

In [27]:
additional_rows = pd.DataFrame(new_rows)

In [29]:
restaurants = pd.concat([editions_df, additional_rows], ignore_index=True)

In [31]:
restaurants.to_csv('data/full_search_restaurants.csv', index=False)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()